In [37]:
import tensorflow as tf

print(tf.__version__)

from tensorflow.python.client import device_lib


1.15.0


In [38]:
# Mask R-CNN
import sys
import numpy as np
import skimage
import time
import os
import cv2

ROOT_DIR = os.path.abspath("./")

sys.path.append(os.path.join(ROOT_DIR, "mrcnn"))
from mrcnn import utils, visualize
from imutils.video import WebcamVideoStream
import mrcnn.model as modellib
from mrcnn.model import mold_image

# Root directory of the project
from samples.coco.coco import CocoConfig



# To find local version# Directory to save logs and trained model
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))

MODEL_DIR = os.path.join(ROOT_DIR, "logs")# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "models", "13.h5")

In [39]:
class InferenceConfig(CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 3

config = InferenceConfig()
config.display()# Create model object in inference mode.


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [ ]:
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)# Define COCO Class names

In [ ]:
class_names = ['BG', 'green_card', 'gold_card']

In [ ]:
# Open webcam at the ID 0
cap = cv2.VideoCapture(0)#Check whether user selected camera is opened successfully.
if not (cap.isOpened()):
    print("Could not open video device")

In [ ]:
# Real-Time Webcam Object Detection
colors = visualize.random_colors(10000)
gentle_grey = (45, 65, 79)
white = (255, 255, 255)

vs = cv2.VideoCapture(0)


SCREEN_NAME = "Real-Time Webcam"
cv2.namedWindow(SCREEN_NAME, cv2.WINDOW_NORMAL)
cv2.setWindowProperty(SCREEN_NAME, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
# Capture frame-by-frame
while True:
    grabbed, frame = vs.read()
    if not grabbed:
        break

    results = model.detect([frame])
    r = results[0]# Run detection

    masked_image = visualize.display_instances(frame, boxes=r['rois'], masks=r['masks'],
                                                     class_ids=r['class_ids'], class_names=class_names,
                                                     scores=r['scores'])
    # change for screen resolution
    masked_image = cv2.resize(masked_image,(1920,1080))
    cv2.imshow(SCREEN_NAME, masked_image)
    cv2.waitKey(1)

# When everything is done, release the camera from video capture
vs.release()
cv2.destroyAllWindows()